# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Лабораторная работа 9

In [11]:
import nltk
import pandas as pd
import random
nltk.download('punkt')
!pip install python-Levenshtein

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [6]:
recipes = pd.read_csv('/content/preprocessed_descriptions.csv') 
recipes =  recipes.dropna(subset=['preprocessed_descriptions']) 
 
words = set() 
nwords = 0 
 
for index in range(recipes.shape[0]): 
    str1 = recipes['preprocessed_descriptions'].iloc[index] 
    lw = nltk.word_tokenize(str1) 
    for item1 in lw: 
        if item1.isalpha() == True: 
            words.add(item1.lower()) 
            nwords+=1 
 
print('Total words:', nwords, 'Unique words:',len(words))
print(words)

Total words: 1058563 Unique words: 30743
{'bananas', 'farms', 'lebon', 'passport', 'burnet', 'knew', 'foodlions', 'jenniam', 'eagle', 'substitue', 'bowies', 'gazelle', 'touches', 'now', 'thepotato', 'sail', 'anniversary', 'dryyou', 'prepareit', 'microwaveable', 'yummynote', 'toastie', 'fictional', 'crouton', 'thisplease', 'hme', 'kladdkaka', 'wether', 'irony', 'grenada', 'backsubstitute', 'hurray', 'sorted', 'johnsonville', 'ciudad', 'dislikes', 'disregard', 'seed', 'nesting', 'wwwckproductscom', 'ways', 'moistyes', 'links', 'parisians', 'halfed', 'oamcthis', 'preheat', 'wwwsouthbeachdietplancom', 'softest', 'gauges', 'newsletter', 'four', 'funky', 'hideout', 'optional', 'lolyou', 'olio', 'hostessing', 'saucepan', 'behr', 'hippychic', 'intoxicating', 'trylooks', 'stwerart', 'connoisseurs', 'leaders', 'weddingnote', 'overwhelm', 'lofas', 'updatewhen', 'unwrapping', 'si', 'gandy', 'nanami', 'antidote', 'goodalmost', 'colourfulprep', 'stirins', 'course', 'vegetarian', 'silks', 'unless', '

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [7]:
import random
from nltk.metrics import edit_distance 

word_pairs = random.sample(words, k=10)
word_pairs = [(word_pairs[i], word_pairs[i+1]) for i in range(0, len(word_pairs), 2)]

for pair in word_pairs:
    dist = edit_distance(pair[0], pair[1])
    print(f"Расстояние редактирования между '{pair[0]}' и '{pair[1]}': {dist}")

Расстояние редактирования между 'obvious' и 'napastyle': 9
Расстояние редактирования между 'nelson' и 'madrid': 6
Расстояние редактирования между 'manufacturing' и 'hatched': 11
Расстояние редактирования между 'falling' и 'signatures': 9
Расстояние редактирования между 'bradford' и 'bergys': 7


<ipython-input-7-5a06a35ce65c>:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  word_pairs = random.sample(words, k=10)


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [12]:
from Levenshtein import distance

def find_closest_words(word, words, k):
  distances = [(w, distance(word, w)) for w in words]
  sorted_distances = sorted(distances, key=lambda x: x[1])
  return [w for w, d in sorted_distances[:k]]
find_closest_words("bananas", words, 4)

['bananas', 'banana', 'bananes', 'kanasas']

### Стемминг, лемматизация

In [13]:
!pip install snowballstemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [17]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import snowballstemmer
stemmer = snowballstemmer.stemmer('english')

df = pd.DataFrame(index=list(words),columns=['stemmed_word', 'normalized_word'])
list_of_words=list(words)
df['stemmed_word']=stemmer.stemWords(list_of_words)
df['normalized_word']=[lemmatizer.lemmatize(w) for w in list_of_words]
df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,stemmed_word,normalized_word
bananas,banana,banana
farms,farm,farm
lebon,lebon,lebon
passport,passport,passport
burnet,burnet,burnet
...,...,...
textureit,textureit,textureit
helmer,helmer,helmer
warns,warn,warns
landed,land,landed


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [18]:
from collections import Counter
word_count = Counter(words)
top_10_words = word_count.most_common(10)
print(top_10_words) 

[('bananas', 1), ('farms', 1), ('lebon', 1), ('passport', 1), ('burnet', 1), ('knew', 1), ('foodlions', 1), ('jenniam', 1), ('eagle', 1), ('substitue', 1)]


### Векторное представление текста

In [23]:
import pandas as pd
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [37]:
recipes = pd.read_csv('/content/preprocessed_descriptions.csv')
recipes =  recipes.dropna(subset=['preprocessed_descriptions'])
recipes = recipes.sample(5)
recipes.reindex()

list1 = []
names = []

vectorizer = TfidfVectorizer()
for index in range(5):
    list1.append(recipes['preprocessed_descriptions'].iloc[index])
    names.append(recipes['name'].iloc[index])
    
X=vectorizer.fit_transform(list1)
a = X.toarray()
for x in range(len(a)):
  print('Числовой вектор для', names[x],'-', a[x],'\n')

Числовой вектор для pasta with lemon zucchini sauce - [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.29416329 0.
 0.         0.         0.23732909 0.23732909 0.         0.
 0.         0.         0.23732909 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.29416329 0.
 0.         0.         0.         0.29416329 0.         0.
 0.29416329 0.         0.         0.         0.         0.
 0.29416329 0.         0.         0.         0.29416329 0.
 0.         0.         0.         0.         0.         0.
 0.23732909 0.         0.         0.         0.23732909 0.
 0.29416329 0.23732909 0.         0.         0.23732909 0.
 0.         0.        ] 

Числовой вектор для petite caramel rolls - [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.55032913 0.         0.         0.44400208 0.    

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [38]:
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

df = pd.DataFrame(columns=names)
row = [0,0,0,0,0]
for index1 in range(5):
    for index2 in range(5):
        w = spatial.distance.cosine(a[index1], a[index2])
        row[index2] = w
    df.loc[len(df)] = row
df.index = names
print(df.to_string())

                                              pasta with lemon zucchini sauce  petite caramel rolls  green chile tortilla pinwheels  sock it to me cake  chicken breasts in sour cream with mushrooms
pasta with lemon zucchini sauce                                      0.000000              0.789251                        0.853559            0.971494                                      0.957559
petite caramel rolls                                                 0.789251              0.000000                        1.000000            1.000000                                      1.000000
green chile tortilla pinwheels                                       0.853559              1.000000                        0.000000            0.987233                                      0.980992
sock it to me cake                                                   0.971494              1.000000                        0.987233            0.000000                                      0.813363
chicken br